In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://statbase.org/datasets/tourism-and-leisure/theme-parks-attendance/?syear=2023&frmreq=0&sproc=proc&filter=&sort="
resp = requests.get(url)
resp.raise_for_status()  # para asegurarnos de que la petición fue exitosa
html = resp.text

In [3]:
soup = BeautifulSoup(html, "html.parser")

# Supongamos que los datos están en un <table>…
table = soup.find("table")
soup = BeautifulSoup(resp.text, "html.parser")

In [4]:
rows = soup.find_all("div", class_="it_row")
all_data = []
for row in rows:
    # dentro de la fila, buscar todas las “celdas”
    cells = row.find_all("div", class_="it_cell")
    row_vals = []
    for c in cells:
        text = c.get_text(strip=True)
        row_vals.append(text)
    all_data.append(row_vals)

# Convertir a dataframe
asistencia_parques = pd.DataFrame(all_data)
print(asistencia_parques.head())

                                                   0         1     2
0                                             Object  share, %  Rank
1  MAGIC KINGDOM THEME PARK AT WALT DISNEY WORLD ...      4.79     1
2      DISNEYLAND PARK AT DISNEYLAND RESORT, ANAHEIM      4.66     2
3                     UNIVERSAL STUDIOS JAPAN, OSAKA      4.33     3
4     TOKYO DISNEYLAND AT TOKYO DISNEY RESORT, TOKYO      4.08     4


In [5]:
asistencia_parques

,0,1,2
0,Object,"share, %",Rank
1,MAGIC KINGDOM THEME PARK AT WALT DISNEY WORLD ...,4.79,1
2,"DISNEYLAND PARK AT DISNEYLAND RESORT, ANAHEIM",4.66,2
3,"UNIVERSAL STUDIOS JAPAN, OSAKA",4.33,3
4,"TOKYO DISNEYLAND AT TOKYO DISNEY RESORT, TOKYO",4.08,4
...,...,...,...
66,"FANTASIALANDIA, SANTIAGO",0.26,63
67,"MUNDO PETAPA, GUATEMALA CITY",0.25,64
68,"THEME PARQUE NACIONAL DEL CAFÉ, QUINDIO",0.23,65
69,"PARQUE DE LA COSTA, BUENOS AIRES",0.17,66


In [6]:
asistencia_parques.columns = ['Nombre del Parque', 'share, %', "Rank"]

In [7]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar las filas de la tabla
rows = soup.find_all('div', class_='it_row')

# Inicializar listas para almacenar los datos
parques = []
ubicaciones = []
for row in rows:
    # Extraer la primera celda que contiene el nombre y ubicación
    cell = row.find('div', class_='it_cell')
    if cell:
        # Obtener el texto de la celda
        texto = cell.get_text(strip=True)
        # Separar el nombre del parque y la ubicación por la coma
        if ',' in texto:
            nombre_parque, ubicacion = texto.rsplit(',', 1)
            parques.append(nombre_parque.strip())
            ubicaciones.append(ubicacion.strip())
        else:
            parques.append(texto)
            ubicaciones.append('Desconocido')

# Crear un DataFrame con los datos
ubicaciones = pd.DataFrame({
    'Nombre del Parque': parques,
    'Ubicación (País)': ubicaciones
})


In [8]:
asistencia_parques

,Nombre del Parque,"share, %",Rank
0,Object,"share, %",Rank
1,MAGIC KINGDOM THEME PARK AT WALT DISNEY WORLD ...,4.79,1
2,"DISNEYLAND PARK AT DISNEYLAND RESORT, ANAHEIM",4.66,2
3,"UNIVERSAL STUDIOS JAPAN, OSAKA",4.33,3
4,"TOKYO DISNEYLAND AT TOKYO DISNEY RESORT, TOKYO",4.08,4
...,...,...,...
66,"FANTASIALANDIA, SANTIAGO",0.26,63
67,"MUNDO PETAPA, GUATEMALA CITY",0.25,64
68,"THEME PARQUE NACIONAL DEL CAFÉ, QUINDIO",0.23,65
69,"PARQUE DE LA COSTA, BUENOS AIRES",0.17,66


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
 


In [10]:
# URL de la página con los datos
url = 'https://statbase.org/datasets/tourism-and-leisure/theme-parks-attendance/'

# Realizar la solicitud HTTP
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar las filas de la tabla
rows = soup.find_all('div', class_='it_row')

# Inicializar listas para almacenar los datos
parques = []
ubicaciones = []

# Iterar sobre cada fila para extraer los datos
for row in rows:
    # Extraer la primera celda que contiene el nombre y ubicación
    cell = row.find('div', class_='it_cell')
    if cell:
        # Obtener el texto de la celda
        texto = cell.get_text(strip=True)
        # Separar el nombre del parque y la ubicación por la coma
        if ',' in texto:
            nombre_parque, ubicacion = texto.rsplit(',', 1)
            parques.append(nombre_parque.strip())
            ubicaciones.append(ubicacion.strip())
        else:
            parques.append(texto)
            ubicaciones.append('Desconocido')

# Crear un DataFrame con los datos
ubicaciones = pd.DataFrame({
    'Nombre del Parque': parques,
    'Ubicación (País)': ubicaciones
})



In [11]:
ubicaciones

,Nombre del Parque,Ubicación (País)
0,Object,Desconocido
1,MAGIC KINGDOM THEME PARK AT WALT DISNEY WORLD ...,LAKE BUENA VISTA
2,DISNEYLAND PARK AT DISNEYLAND RESORT,ANAHEIM
3,UNIVERSAL STUDIOS JAPAN,OSAKA
4,TOKYO DISNEYLAND AT TOKYO DISNEY RESORT,TOKYO
...,...,...
66,FANTASIALANDIA,SANTIAGO
67,MUNDO PETAPA,GUATEMALA CITY
68,THEME PARQUE NACIONAL DEL CAFÉ,QUINDIO
69,PARQUE DE LA COSTA,BUENOS AIRES


In [12]:
ubicaciones['Nombre del Parque'] = ubicaciones['Nombre del Parque'].str.strip().str.lower()
asistencia_parques['Nombre del Parque'] = asistencia_parques['Nombre del Parque'].str.strip().str.lower()

In [13]:
parques = pd.merge(
    ubicaciones,
    asistencia_parques,
    on='Nombre del Parque',
    how='outer'
)


In [14]:
parques

,Nombre del Parque,Ubicación (País),"share, %",Rank
0,alton towers,STAFFORDSHIRE,NaN,NaN
1,"alton towers, staffordshire",NaN,0.64,50
2,beto carrero world,SANTA CATARINA,NaN,NaN
3,"beto carrero world, santa catarina",NaN,0.75,47
4,busch gardens tampa bay,TAMPA,NaN,NaN
...,...,...,...,...
134,"warner bros abu dhabi, abu dhabi",NaN,0.47,56
135,zhengzhou fantawild adventure,ZHENGZHOU,NaN,NaN
136,"zhengzhou fantawild adventure, zhengzhou",NaN,1.2,26
137,zigong fantawild dino kingdom,ZIGONG,NaN,NaN


In [15]:
# Ordenar por 'Share' de mayor a menor
ordenado = parques.sort_values(by='share, %', ascending=False)

# Quedarte con las primeras 10 filas (top 10)
top10_paises = ordenado.head(10).reset_index(drop=True)




In [16]:
top10_paises

,Nombre del Parque,Ubicación (País),"share, %",Rank
0,object,Desconocido,"share, %",Rank
1,magic kingdom theme park at walt disney world ...,NaN,4.79,1
2,"disneyland park at disneyland resort, anaheim",NaN,4.66,2
3,"universal studios japan, osaka",NaN,4.33,3
4,"tokyo disneyland at tokyo disney resort, tokyo",NaN,4.08,4
5,"shanghai disneyland at shanghai disney resort,...",NaN,3.79,5
6,"chimelong ocean kingdom, hengqin",NaN,3.38,6
7,"tokyo disneysea at tokyo disney resort, tokyo",NaN,3.35,7
8,"epcot at walt disney world, lake buena vista",NaN,3.24,8
9,"disneyland park at disneyland paris, marne-la-...",NaN,2.81,9


In [17]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://blog.europa.jobs/es/art-salario-minimo-en-europa-en-2025/"

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

# Encontrar todas las tablas de la página
tables = soup.find_all('table')
print(f"Se encontraron {len(tables)} tablas.")

dfs = []

for i, table in enumerate(tables):
    rows = table.find_all('tr')
    
    # Obtener encabezados si existen
    headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
    
    data = []
    for row in rows[1:]:
        cols = row.find_all('td')
        if len(cols) == 0:
            continue
        data.append([col.get_text(strip=True) for col in cols])
    
    salario_minimo = pd.DataFrame(data, columns=headers)
    dfs.append(salario_minimo)
    
  

# Ahora dfs es una lista con todos los DataFrames de cada tabla


Se encontraron 4 tablas.


In [20]:
dfs[0]

,Grupo I,Salario mínimo
0,Bulgaria,"550,66 EUR"
1,Hungría,"773,90/883,82 EUR"
2,Letonia,740 EUR


In [21]:
dfs[0].rename(columns={
    "Grupo I":"País",
}, inplace=True)

In [22]:
dfs[1]

,Grupo II,Salario mínimo
0,Rumanía,"814,49 EUR"
1,Grecia,968 EUR
2,Croacia,970 EUR
3,República Checa,"823,3 EUR"
4,Estonia,886 EUR
5,Malta,"961,05 EUR"
6,Eslovaquia,816 EUR


In [23]:
dfs[1].rename(columns={
    "Grupo II":"País",
}, inplace=True)

In [24]:
dfs[2]

,Grupo III,Salario mínimo
0,Polonia,"1085,57 EUR"
1,Portugal,1015 EUR
2,Lituania,1038 EUR
3,Eslovenia,"1253,36 EUR"
4,España,1323 EUR
5,Francia,"1801,80 EUR"


In [25]:
dfs[2].rename(columns={
    "Grupo III":"País",
}, inplace=True)

In [26]:
dfs[3]

,Grupo IV,Salario mínimo
0,Alemania,2222 EUR
1,Bélgica,"2029,88 EUR"
2,Países Bajos,2437 EUR
3,Irlanda,"2281,50 EUR"
4,Luxemburgo,"2637,77 EUR"


In [27]:
dfs[3].rename(columns={
    "Grupo IV":"País",
}, inplace=True)

In [28]:
salario_minimo_europa = pd.concat(dfs, ignore_index=True, sort=False)


In [29]:
salario_minimo_europa

,País,Salario mínimo
0,Bulgaria,"550,66 EUR"
1,Hungría,"773,90/883,82 EUR"
2,Letonia,740 EUR
3,Rumanía,"814,49 EUR"
4,Grecia,968 EUR
5,Croacia,970 EUR
6,República Checa,"823,3 EUR"
7,Estonia,886 EUR
8,Malta,"961,05 EUR"
9,Eslovaquia,816 EUR


In [30]:
salario_minimo_europa = salario_minimo_europa.sort_values(by="País", ascending=True)


In [31]:
salario_minimo_europa

,País,Salario mínimo
16,Alemania,2222 EUR
0,Bulgaria,"550,66 EUR"
17,Bélgica,"2029,88 EUR"
5,Croacia,970 EUR
9,Eslovaquia,816 EUR
13,Eslovenia,"1253,36 EUR"
14,España,1323 EUR
7,Estonia,886 EUR
15,Francia,"1801,80 EUR"
4,Grecia,968 EUR


In [32]:
# 1. Limpiar la columna de salarios (si tiene símbolos o texto como €)
salario_minimo_europa['Salario mínimo'] = (
    salario_minimo_europa['Salario mínimo']
    .str.replace(r'[^\d,.-]', '', regex=True)   # elimina € u otros caracteres
    .str.replace(',', '.')                     # cambia coma decimal por punto
)

# 2. Convertir a float (forzamos errores como NaN si hay algo raro)
salario_minimo_europa['Salario mínimo'] = pd.to_numeric(salario_minimo_europa['Salario mínimo'], errors='coerce')

# 3. Ordenar de mayor a menor
último_salario = salario_minimo_europa.sort_values(by='Salario mínimo', ascending=False)

In [33]:
último_salario

,País,Salario mínimo
20,Luxemburgo,2637.77
18,Países Bajos,2437.00
19,Irlanda,2281.50
16,Alemania,2222.00
17,Bélgica,2029.88
15,Francia,1801.80
14,España,1323.00
13,Eslovenia,1253.36
10,Polonia,1085.57
12,Lituania,1038.00


In [34]:
top_10_salario = último_salario.head(10)

In [35]:
top_10_salario

,País,Salario mínimo
20,Luxemburgo,2637.77
18,Países Bajos,2437.00
19,Irlanda,2281.50
16,Alemania,2222.00
17,Bélgica,2029.88
15,Francia,1801.80
14,España,1323.00
13,Eslovenia,1253.36
10,Polonia,1085.57
12,Lituania,1038.00


In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página que quieres extraer
url = "https://datosmacro.expansion.com/paises/grupos/zona-euro"

# Realizar la petición HTTP
resp = requests.get(url)
resp.raise_for_status()  # asegurarnos que la petición fue exitosa

# Parsear con BeautifulSoup
soup = BeautifulSoup(resp.text, 'html.parser')

# Encontrar la tabla (o las tablas) relevantes
# Aquí suponemos que la tabla está identificada como <table>
tables = soup.find_all('table')
print(f"Se han encontrado {len(tables)} tablas en la página.")

# Supongamos que la tabla que quieres es la primera
table = tables[0]

# Extraer los encabezados
header_cells = table.find('thead').find_all('th')
headers = [cell.get_text(strip=True) for cell in header_cells]

# Extraer las filas de datos
data = []
for row in table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    if not cols:
        continue
    data.append([col.get_text(strip=True) for col in cols])

# Crear un DataFrame con pandas
pais_poblacion = pd.DataFrame(data, columns=headers)

# Ver el DataFrame resultante
print(pais_poblacion.head())




Se han encontrado 1 tablas en la página.
         Países   Población     PIB anual     PIB anual PIB Per Capita  \
0  Alemania [+]  83.577.140  4.328.970 M€  4.685.593 M$       51.830 €   
1   Austria [+]   9.197.213    494.088 M€    534.791 M$       53.830 €   
2   Bélgica [+]  11.900.123    620.272 M€    664.564 M$       52.370 €   
3    Chipre [+]     979.865     33.568 M€     36.322 M$       35.730 €   
4   Croacia [+]   3.874.350     85.905 M€     92.982 M$       22.200 €   

  PIB Per Capita    IDH Deuda total (M.€) Deuda total (M.$) Deuda (%PIB)  \
0       56.087 $  0,950         2.688.879         2.972.835       62,50%   
1       57.208 $  0,926           394.131           423.300       81,80%   
2       56.221 $  0,942           642.875           695.626      104,70%   
3       38.802 $  0,907            21.828            23.640       65,00%   
4       24.025 $  0,878            49.283            53.327       57,60%   

  Déficit (%PIB)  
0         -2,80%  
1         -4,70%  


In [50]:
pais_poblacion["Países"] = pais_poblacion["Países"].str.replace("[+]", "", regex=False)


In [51]:
pais_poblacion

,Países,Población,PIB anual,PIB anual,PIB Per Capita,PIB Per Capita,IDH,Deuda total (M.€),Deuda total (M.$)
0,Alemania,83.577.140,4.328.970 M€,4.685.593 M$,51.830 €,56.087 $,"0,950",2.688.879,2.972.835
1,Austria,9.197.213,494.088 M€,534.791 M$,53.830 €,57.208 $,"0,926",394.131,423.300
2,Bélgica,11.900.123,620.272 M€,664.564 M$,52.370 €,56.221 $,"0,942",642.875,695.626
3,Chipre,979.865,33.568 M€,36.322 M$,35.730 €,38.802 $,"0,907",21.828,23.640
4,Croacia,3.874.350,85.905 M€,92.982 M$,22.200 €,24.025 $,"0,878",49.283,53.327
5,Eslovaquia,5.419.451,130.208 M€,141.776 M$,23.850 €,26.127 $,"0,855",77.648,81.118
6,Eslovenia,2.130.850,67.418 M€,72.972 M$,31.700 €,34.346 $,"0,926",44.895,48.579
7,España,49.315.949,1.594.330 M€,1.725.672 M$,32.630 €,35.151 $,"0,911",1.620.573,1.753.580
8,Estonia,1.369.995,39.848 M€,43.130 M$,28.990 €,31.419 $,"0,899",9.327,10.092
9,Finlandia,5.635.971,275.963 M€,298.697 M$,49.100 €,53.286 $,"0,942",226.701,245.304


In [56]:
pais_poblacion.drop(columns=['PIB anual', 'PIB anual', 'PIB Per Capita', 'PIB Per Capita', 'IDH', 'Deuda total (M.€)', 'Deuda total (M.$)'], inplace=True)


In [57]:
pais_poblacion

,Países,Población
0,Alemania,83.577.140
1,Austria,9.197.213
2,Bélgica,11.900.123
3,Chipre,979.865
4,Croacia,3.874.350
5,Eslovaquia,5.419.451
6,Eslovenia,2.130.850
7,España,49.315.949
8,Estonia,1.369.995
9,Finlandia,5.635.971


In [58]:
pais_poblacion.rename(columns={
    "Países": "País",
}, inplace=True)


In [59]:
import unicodedata

def limpiar_pais(nombre):
    return (
        str(nombre)
        .strip()  # quita espacios al inicio/final
        .replace("+", "")  # elimina '+'
        .replace("\xa0", " ")  # reemplaza espacio no separable
        .lower()  # todo minúsculas
        .title()  # primera letra en mayúscula
        .replace("’", "'")  # comillas raras
        .replace("´", "'")  # tildes incorrectas
        .replace(",", "")  # elimina comas
        .replace(".", "")  # elimina puntos
    )

# Aplicar a ambos
salario_minimo_europa["País"] = salario_minimo_europa["País"].apply(limpiar_pais)
pais_poblacion["País"] = pais_poblacion["País"].apply(limpiar_pais)


In [60]:
poblacion_salario = pd.merge(salario_minimo_europa, pais_poblacion, on="País", how="outer")
poblacion_salario = poblacion_salario.groupby("País").agg({
    "Salario mínimo": "first",
    "Población": "first"
}).reset_index()



In [61]:
poblacion_salario = poblacion_salario.sort_values(by="Salario mínimo", ascending=False)


In [62]:
top_poblacion_salario = poblacion_salario.head(15)

In [63]:
top_poblacion_salario

,País,Salario mínimo,Población
18,Luxemburgo,2637.77,681.973
20,Países Bajos,2437.00,18.044.027
14,Irlanda,2281.50,5.439.898
0,Alemania,2222.00,83.577.140
3,Bélgica,2029.88,11.900.123
11,Francia,1801.80,68.635.943
8,España,1323.00,49.315.949
7,Eslovenia,1253.36,2.130.850
21,Polonia,1085.57,None
17,Lituania,1038.00,2.890.664


In [64]:
import pandas as pd

# Datos de los parques de atracciones
data = {
    "Parque Temático": [
        "Disneyland Park Paris",
        "Europa-Park",
        "Walt Disney Studios Park",
        "De Efteling",
        "Tivoli Gardens",
        "PortAventura World",
        "Gardaland",
        "Parc Astérix",
        "Puy du Fou",
        "Legoland Windsor",
        "Alton Towers",
        "Legoland Billund",
        "Liseberg"
    ],
    "País": [
        "Francia",
        "Alemania",
        "Francia",
        "Países Bajos",
        "Dinamarca",
        "España",
        "Italia",
        "Francia",
        "Francia",
        "Reino Unido",
        "Reino Unido",
        "Dinamarca",
        "Suecia"
    ],
    "Visitantes (millones)": [
        10.4,
        6.0,
        5.7,
        5.56,
        4.03,
        3.98,
        3.07,
        2.81,
        2.5,
        2.42,
        2.35,
        2.35,
        2.3
    ]
}

# Crear DataFrame
parques_europa= pd.DataFrame(data)

# Guardar en archivo CSV
parques_europa.to_csv("parques_atracciones_europa.csv", index=False)

print("Archivo 'parques_atracciones_europa.csv' creado correctamente.")


Archivo 'parques_atracciones_europa.csv' creado correctamente.


In [65]:
parques_europa

,Parque Temático,País,Visitantes (millones)
0,Disneyland Park Paris,Francia,10.40
1,Europa-Park,Alemania,6.00
2,Walt Disney Studios Park,Francia,5.70
3,De Efteling,Países Bajos,5.56
4,Tivoli Gardens,Dinamarca,4.03
5,PortAventura World,España,3.98
6,Gardaland,Italia,3.07
7,Parc Astérix,Francia,2.81
8,Puy du Fou,Francia,2.50
9,Legoland Windsor,Reino Unido,2.42


In [69]:
parques_completo = pd.merge(top_poblacion_salario, parques_europa, on="País", how="outer")
parques_completo = parques_completo.groupby("País").agg({
    "Población": "first",
    "Salario mínimo": "first",
    "Parque Temático": "first",
    "Visitantes (millones)": "first"
}).reset_index()

In [70]:
parques_completo

,País,Población,Salario mínimo,Parque Temático,Visitantes (millones)
0,Alemania,83.577.140,2222.00,Europa-Park,6.00
1,Bélgica,11.900.123,2029.88,None,NaN
2,Croacia,3.874.350,970.00,None,NaN
3,Dinamarca,None,NaN,Tivoli Gardens,4.03
4,Eslovenia,2.130.850,1253.36,None,NaN
5,España,49.315.949,1323.00,PortAventura World,3.98
6,Estonia,1.369.995,886.00,None,NaN
7,Francia,68.635.943,1801.80,Disneyland Park Paris,10.40
8,Grecia,10.409.547,968.00,None,NaN
9,Irlanda,5.439.898,2281.50,None,NaN


In [109]:
import pandas as pd

ruta = 'C:/Users/noeli/Downloads/xls0009914_i.csv'

menores_15 = pd.read_csv(
    ruta,
    sep=';',
    encoding='latin1',
    skiprows=4  # ajustar si es necesario
)

print(menores_15.head())
print(menores_15.columns)


      EU 27  15,4 15,4.1  15,3 15,3.1 15,3.2  15,2 15,2.1 15,2.2  15,1    15  \
0   Germany  13,4   13,3  13,2   13,2   13,2  13,4   13,5   13,6  13,7  13,8   
1   Austria  14,6   14,4  14,3   14,3   14,3  14,4   14,4   14,4  14,4  14,4   
2   Belgium    17     17    17     17     17    17     17   16,9  16,9  16,8   
3  Bulgaria  13,4   13,6  13,7   13,9     14  14,1   14,2   14,4  14,4  14,4   
4    Cyprus  16,5   16,4  16,3   16,4   16,4  16,3   16,2   16,1    16    16   

   15.1  14,9  Unnamed: 13  
0  13,9  14,1          NaN  
1  14,4  14,4          NaN  
2  16,6  16,5          NaN  
3  14,5  14,2          NaN  
4  16,1  16,1          NaN  
Index(['EU 27', '15,4', '15,4.1', '15,3', '15,3.1', '15,3.2', '15,2', '15,2.1',
       '15,2.2', '15,1', '15', '15.1', '14,9', 'Unnamed: 13'],
      dtype='object')


In [110]:
menores_15 = menores_15.rename(columns={"14,9": "Menores 15(%)"})
menores_15 = menores_15.rename(columns={"EU 27": "País"})
menores_15.drop(columns=["15,2.1", "15,2.2", "15,1", "15", "15.1"], inplace=True)
menores_15.drop(columns=["15,4", "15,4.1", "15,3", "15,3.1", "15,3.2", "15,2"], inplace=True)
menores_15.drop(columns=["Unnamed: 13"],inplace=True)

In [111]:
pip install googletrans==4.0.0-rc1


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [112]:
from googletrans import Translator
import pandas as pd

translator = Translator()

# Supongamos que df es tu DataFrame y 'column_name' la columna en inglés
menores_15['País'] = menores_15['País'].apply(lambda x: translator.translate(x, src='en', dest='es').text)


In [121]:
parques_menores15 = pd.merge(parques_completo, menores_15, on='País', how='inner')
col = 'Menores 15(%)'
pos = 2  # posición deseada (0 = primera columna)

cols = list(parques_menores15.columns)
cols.insert(pos, cols.pop(cols.index(col)))  # mueve la columna

parques_menores15 = parques_menores15[cols]


In [124]:
parques_menores15["Población"] = parques_menores15["Población"].fillna("Desconocido")
parques_menores15["Salario mínimo"] = parques_menores15["Salario mínimo"].fillna("Desconocido")
parques_menores15["Parque Temático"] = parques_menores15["Parque Temático"].fillna("No tiene")
parques_menores15["Visitantes (millones)"] = parques_menores15["Visitantes (millones)"].fillna(" - ")


In [125]:
parques_menores15

,País,Población,Menores 15(%),Salario mínimo,Parque Temático,Visitantes (millones)
0,Alemania,83.577.140,"14,1",2222.0,Europa-Park,6.0
1,Bélgica,11.900.123,"16,5",2029.88,No tiene,-
2,Croacia,3.874.350,"14,3",970.0,No tiene,-
3,Dinamarca,Desconocido,16,Desconocido,Tivoli Gardens,4.03
4,Eslovenia,2.130.850,15,1253.36,No tiene,-
5,España,49.315.949,"13,6",1323.0,PortAventura World,3.98
6,Estonia,1.369.995,"16,4",886.0,No tiene,-
7,Francia,68.635.943,"17,3",1801.8,Disneyland Park Paris,10.4
8,Grecia,10.409.547,"13,4",968.0,No tiene,-
9,Irlanda,5.439.898,"19,3",2281.5,No tiene,-


In [ ]:
#hacer todo esto de las provincias españolas
#poblacion, salario minimo, porcentaje de jovenes, si tienen ya un parque o no, numero de visitantes (extranjeros o locales)

In [42]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


In [ ]:
poblacion_españa = pd.read_csv('C:/Users/noeli/Downloads/2852.csv', sep=';') 

print(poblacion_españa.columns)

print(poblacion_españa['Periodo'].dtype)

#Quedarme solo con 2021
población_españa = poblacion_españa[poblacion_españa['Periodo'] == 2021]
poblacion_españa['Periodo'] = pd.to_numeric(poblacion_españa['Periodo'], errors='coerce')
poblacion_españa_filtrada = poblacion_españa[poblacion_españa['Periodo'] == 2021]
print(poblacion_españa_filtrada)


Index(['Provincias', 'Sexo', 'Periodo', 'Total'], dtype='object')
int64
       Provincias     Sexo  Periodo       Total
0           Total    Total     2021  47.385.107
26          Total  Hombres     2021  23.222.953
52          Total  Mujeres     2021  24.162.154
78    02 Albacete    Total     2021     386.464
104   02 Albacete  Hombres     2021     193.205
...           ...      ...      ...         ...
4004     51 Ceuta  Hombres     2021      42.208
4030     51 Ceuta  Mujeres     2021      41.309
4056   52 Melilla    Total     2021      86.261
4082   52 Melilla  Hombres     2021      43.603
4108   52 Melilla  Mujeres     2021      42.658

[159 rows x 4 columns]


In [45]:
definitivo = poblacion_españa_filtrada.pivot(index='Provincias', columns='Sexo', values='Total').reset_index()


In [46]:
definitivo

Sexo,Provincias,Hombres,Mujeres,Total
0,01 Araba/Álava,164.252,169.374,333.626
1,02 Albacete,193.205,193.259,386.464
2,03 Alicante/Alacant,932.018,949.744,1.881.762
3,04 Almería,373.136,358.656,731.792
4,05 Ávila,79.562,78.859,158.421
5,06 Badajoz,331.025,338.918,669.943
6,"07 Balears, Illes",584.853,588.155,1.173.008
7,08 Barcelona,2.791.250,2.923.480,5.714.730
8,09 Burgos,177.920,178.135,356.055
9,10 Cáceres,192.951,196.607,389.558


In [47]:
import re
definitivo["Provincias"] = definitivo["Provincias"].str.replace(r"^\d+\s*", "", regex=True)


In [183]:
pip install dateparser

     -------------------------------------- 315.5/315.5 kB 1.8 MB/s eta 0:00:00
     -------------------------------------- 277.6/277.6 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [184]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")



In [185]:
comunidades_autonomas = pd.read_csv("C:/Users/noeli/Downloads/58480 (1).csv",sep=';')

C:\Users\noeli\AppData\Local\Temp\ipykernel_31004\2501422243.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  comunidades_autonomas = pd.read_csv("C:/Users/noeli/Downloads/58480 (1).csv",sep=';')


In [167]:
comunidades_autonomas

,Sexo,Valores simples de edad,Provincias,Periodo,Total
0,Total,Todas las edades,Total Nacional,1 de enero de 1971,34.040.642
1,Total,Todas las edades,Total Nacional,1 de julio de 1971,34.216.856
2,Total,Todas las edades,Total Nacional,1 de enero de 1972,34.408.338
3,Total,Todas las edades,Total Nacional,1 de julio de 1972,34.595.886
4,Total,Todas las edades,Total Nacional,1 de enero de 1973,34.800.600
...,...,...,...,...,...
1703203,Mujeres,85 y más años,52 Melilla,1 de julio de 2020,829
1703204,Mujeres,85 y más años,52 Melilla,1 de enero de 2021,827
1703205,Mujeres,85 y más años,52 Melilla,1 de julio de 2021,835
1703206,Mujeres,85 y más años,52 Melilla,1 de enero de 2022,831


In [187]:
import pandas as pd
import dateparser
# Función para convertir fechas con dateparser
def convertir_fecha_dp(fecha_str):
    if pd.isna(fecha_str):
        return pd.NaT
    return dateparser.parse(fecha_str, languages=['es'])

# Aplica la función a la columna 'Periodo'
comunidades_autonomas['Periodo_dt'] = comunidades_autonomas['Periodo'].apply(convertir_fecha_dp)

# Filtra solo filas con fechas en 2022
filtrado_2022 = comunidades_autonomas[comunidades_autonomas['Periodo_dt'].dt.year == 2022]

# Opcional: resetear índice para comodidad
filtrado_2022 = filtrado_2022.reset_index(drop=True)

# Muestra el resultado
print(filtrado_2022)


KeyboardInterrupt: 

In [180]:
comunidades_autonomas['Periodo_dt'] = comunidades_autonomas['Periodo'].apply(convertir_fecha)
filtrado_2022 = comunidades_autonomas[comunidades_autonomas['Periodo_dt'].dt.year == 2022]
print(filtrado_2022)


Empty DataFrame
Columns: [Sexo, Valores simples de edad, Provincias, Periodo, Total, Periodo_dt]
Index: []


In [173]:
print(comunidades_autonomas.columns)
print(comunidades_autonomas['Periodo'].head(10))
comunidades_autonomas['Periodo_dt'] = comunidades_autonomas['Periodo'].apply(convertir_fecha)
fecha_obj = pd.Timestamp('2022-01-01')
comunidad_autonoma = comunidades_autonomas[comunidades_autonomas['Periodo_dt'] == fecha_obj]
print(comunidad_autonoma)


Index(['Sexo', 'Valores simples de edad', 'Provincias', 'Periodo', 'Total'], dtype='object')
0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
5   NaT
6   NaT
7   NaT
8   NaT
9   NaT
Name: Periodo, dtype: datetime64[ns]
Empty DataFrame
Columns: [Sexo, Valores simples de edad, Provincias, Periodo, Total, Periodo_dt]
Index: []


In [175]:
comunidades_autonomas['Periodo'] = pd.to_datetime(comunidades_autonomas['Periodo'], errors='coerce')
fecha_obj = pd.Timestamp('2022-01-01')
comunidad_autonoma = comunidades_autonomas[comunidades_autonomas['Periodo'] == '2022-01-01']
filtrado_2022 = comunidades_autonomas[
    (comunidades_autonomas['Periodo_dt'].dt.year == 2022)
]
print(filtrado_2022)


Empty DataFrame
Columns: [Sexo, Valores simples de edad, Provincias, Periodo, Total, Periodo_dt]
Index: []


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL del artículo
url = "https://www.dataestur.es/blog/transporte-aereo-en-espana-balance-turistico-del-verano-2025/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encuentra la tabla correcta (puedes ajustar el selector si cambia)
tables = soup.find_all('table')
# Vamos a asumir que la tabla que quieres es la primera (ajusta si necesario)
target_table = tables[0]  # Cambia el índice si hay más de una tabla

# Extraer cabeceras
headers = [th.get_text(strip=True) for th in target_table.find_all('th')]

# Extraer filas
rows = []
for tr in target_table.find_all('tr')[1:]:
    cells = [td.get_text(strip=True) for td in tr.find_all(['td'])]
    if cells:
        rows.append(cells)

# Crear DataFrame
aeropuertos = pd.DataFrame(rows, columns=headers)

print(aeropuertos)


In [ ]:
import pandas as pd

# Ruta del archivo Excel
archivo = "C:/Users/noeli/Downloads/9.-Estadisticas_Septiembre_2025.xlsx"

hoja = 0  

# Leer solo el rango B6:D56
df = pd.read_excel(
    archivo,
    sheet_name=hoja,
    usecols="B:D",
    skiprows=5, 
    nrows=51      
)


In [21]:
df_nuevo = df.drop(columns={"% Inc 2025 /s 2024"})


In [97]:
df_nuevo.head(10)

,Aeropuertos,Total
0,NaN,NaN
1,ADOLFO SUÁREZ MADRID-BARAJAS,5922644.0
2,BARCELONA-EL PRAT J.T.,5279003.0
3,PALMA DE MALLORCA,4113337.0
4,MÁLAGA-COSTA DEL SOL,2626561.0
5,ALICANTE-ELCHE MIGUEL HDEZ.,1907274.0
6,IBIZA,1227406.0
7,GRAN CANARIA,1186060.0
8,VALENCIA,1071198.0
9,TENERIFE SUR,996795.0


In [196]:
import pandas as pd
aeropuertos_comunidades = pd.read_excel("C:/Users/noeli/Downloads/aeropuertos_comerciales_españa.xlsx")

In [202]:
aeropuertos1 = aeropuertos_comunidades.drop(columns=["Código IATA", "Ciudad principal", "Comunidad Autónoma"])
aeropuertos2 = aeropuertos1.rename(columns={"Provincia": "Provincias"})

In [33]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


url = "https://sede.agenciatributaria.gob.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/mercado/2022/jrubikf1b31c8858155bdb0bed1e4e7d45a9e38faf2d78d.html"


resp = requests.get(url)
resp.encoding = resp.apparent_encoding 
html = resp.text


soup = BeautifulSoup(html, "html.parser")


table = soup.find("table")


renta = pd.read_html(str(table))[0]




C:\Users\noeli\AppData\Local\Temp\ipykernel_31004\537877979.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  renta = pd.read_html(str(table))[0]


In [58]:
renta_actualizada = renta.rename(columns={"Unnamed: 0": "Provincias"})

In [59]:
renta_actualizada.drop(columns=["Asalariados", "Percepciones por persona", "Salarios"])

,Provincias,Salario Medio Anual
0,Total,22.781
1,Andalucía,18.503
2,Almería,16.962
3,Cádiz,18.988
4,Córdoba,17.580
5,Granada,18.658
6,Huelva,16.190
7,Jaén,16.012
8,Málaga,19.443
9,Sevilla,19.851


In [75]:
renta_actualizada2 = renta_actualizada.sort_values(by="Provincias").reset_index(drop=True)


In [93]:
provincias_salarios = pd.merge(definitivo, renta_actualizada2, on="Provincias", how="outer")


In [94]:
provincias_salarios2 = provincias_salarios.drop(columns=["Asalariados", "Percepciones por persona", "Salarios"])


In [95]:
provincias_salarios2.rename(columns={"Total": "Personas Totales"})

,Provincias,Hombres,Mujeres,Personas Totales,Salario Medio Anual
0,A Coruña,NaN,NaN,NaN,23.456
1,Albacete,193.205,193.259,386.464,19.680
2,Alicante,NaN,NaN,NaN,19.112
3,Alicante/Alacant,932.018,949.744,1.881.762,NaN
4,Almería,373.136,358.656,731.792,16.962
...,...,...,...,...,...
67,Valencia/València,1.267.961,1.321.351,2.589.312,NaN
68,Valladolid,252.843,266.518,519.361,23.407
69,Zamora,83.551,85.174,168.725,19.254
70,Zaragoza,473.354,494.098,967.452,23.346


In [205]:
local = pd.merge(provincias_salarios2, aeropuertos2, on= "Provincias", how= "outer" )

In [206]:
local

,Provincias,Hombres,Mujeres,Total,Salario Medio Anual,Aeropuerto
0,A Coruña,NaN,NaN,NaN,23.456,Aeropuerto de A Coruña
1,Albacete,193.205,193.259,386.464,19.680,NaN
2,Alicante,NaN,NaN,NaN,19.112,Aeropuerto de Alicante-Elche Miguel Hernández
3,Alicante/Alacant,932.018,949.744,1.881.762,NaN,NaN
4,Almería,373.136,358.656,731.792,16.962,Aeropuerto de Almería
...,...,...,...,...,...,...
77,Valencia/València,1.267.961,1.321.351,2.589.312,NaN,NaN
78,Valladolid,252.843,266.518,519.361,23.407,Aeropuerto de Valladolid
79,Zamora,83.551,85.174,168.725,19.254,NaN
80,Zaragoza,473.354,494.098,967.452,23.346,Aeropuerto de Zaragoza


In [220]:
import pandas as pd

In [222]:
import pandas as pd

edad2 = pd.read_excel('C:/Users/noeli/Downloads/03002.xlsx')



TypeError: 'str' object is not callable

In [218]:
edad2

'C:/Users/noeli/Downloads/03002.xlsx'

In [82]:
edad = pd.read_csv("C:/Users/noeli/Downloads/58480 (1).csv",sep=';')

C:\Users\noeli\AppData\Local\Temp\ipykernel_31004\2416776035.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  edad = pd.read_csv("C:/Users/noeli/Downloads/58480 (1).csv",sep=';')
